In [1]:
import json

In [20]:
with open('../data/raw/v2_Annotations_Val_mscoco/v2_mscoco_val2014_annotations.json') as fp:
    ans = json.load(fp)

In [3]:
with open('../data/raw/v2_Questions_Val_mscoco/v2_OpenEnded_mscoco_val2014_questions.json') as fp:
    ques = json.load(fp)

In [19]:
ans['annotations']

TypeError: list indices must be integers or slices, not str

In [21]:
answers = [[a['answer'] for a in ans_dict['answers']] for ans_dict in ans['annotations']]

In [22]:
import re
# this is used for normalizing questions
_special_chars = re.compile('[^a-z0-9 ]*')

# these try to emulate the original normalization scheme for answers
_period_strip = re.compile(r'(?!<=\d)(\.)(?!\d)')
_comma_strip = re.compile(r'(\d)(,)(\d)')
_punctuation_chars = re.escape(r';/[]"{}()=+\_-><@`,?!')
_punctuation = re.compile(r'([{}])'.format(re.escape(_punctuation_chars)))
_punctuation_with_a_space = re.compile(r'(?<= )([{0}])|([{0}])(?= )'.format(_punctuation_chars))

In [23]:
def process_punctuation(s):
    # the original is somewhat broken, so things that look odd here might just be to mimic that behaviour
    # this version should be faster since we use re instead of repeated operations on str's
    if _punctuation.search(s) is None:
        return s
    s = _punctuation_with_a_space.sub('', s)
    if re.search(_comma_strip, s) is not None:
        s = s.replace(',', '')
    s = _punctuation.sub(' ', s)
    s = _period_strip.sub('', s)
    return s.strip()

In [24]:
def x():
    for answer_list in answers:
        yield list(map(process_punctuation, answer_list))

In [25]:
list(x())

[['down',
  'down',
  'at table',
  'skateboard',
  'down',
  'table',
  'down',
  'down',
  'down',
  'down'],
 ['spectating',
  'watching',
  'watching',
  'watching',
  'watching',
  'watching',
  'watching',
  'watching',
  'watching',
  'watching'],
 ['table',
  'table',
  'table',
  'picnic table',
  'picnic table',
  'picnic table',
  'picnic table',
  'picnic table',
  'skateboard',
  'picnic table'],
 ['foodiebakercom',
  'foodiebakercom',
  'foodiebaker',
  'foodiebakercom',
  'foodiebakercom',
  'http:  foodiebakercom',
  'foodiebakercom',
  'foodiebakercom',
  'foodiebakercom',
  'foodiebaker'],
 ['no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no'],
 ['yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes'],
 ['chopsticks',
  'chopsticks',
  'chopsticks',
  'chopsticks',
  'chopsticks',
  'shrimp',
  'chopsticks',
  'chopsticks',
  'chopsticks',
  'chopsticks spoon'],
 ['crossing it',
  'walking',
  'walking',
  'crossing',
  'crossing road',
  'walking

In [ ]:
answers

In [ ]:
ques['questions']

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image

In [ ]:
def get_transform(target_size, central_fraction=1.0):
    return transforms.Compose([
        transforms.Resize((target_size,target_size)),
#         transforms.Scale(int(target_size / central_fraction)),
#         transforms.RandomCrop(target_size),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225]),
    ])

In [ ]:
transform = get_transform(448)

In [ ]:
img = Image.open('test.jpg')

In [ ]:
trans_img = transform(img)

In [ ]:
trans_img.shape

In [ ]:
transforms.ToPILImage()(trans_img)

In [ ]:
img

In [ ]:
import re,sys,os
import unicodedata
import numpy as np
from torch.utils.data.dataset import Dataset

In [ ]:
ans['annotations'][2]

In [ ]:
ques['questions'][1]

In [37]:
import pandas as pd
import numpy as np

In [7]:
answer = pd.DataFrame(ans['annotations'])
question = pd.DataFrame(ques['questions'])

In [8]:
d = question.merge(answer,how="left",on=['question_id','image_id'])

In [52]:
d[d['answer_type']=='number']

,image_id,question,question_id,answer_type,answers,multiple_choice_answer,question_type
8,393226,How many photo's can you see?,393226001,number,"[{'answer': '1', 'answer_confidence': 'maybe',...",1,how many
15,131089,How many are playing ball?,131089002,number,"[{'answer': '1', 'answer_confidence': 'yes', '...",1,how many
19,262162,How many beds?,262162001,number,"[{'answer': '1', 'answer_confidence': 'yes', '...",1,how many
21,262162,How many pictures are there?,262162003,number,"[{'answer': '7', 'answer_confidence': 'yes', '...",7,how many
23,262162,How many sources of light are there?,262162005,number,"[{'answer': '1', 'answer_confidence': 'maybe',...",1,how many
36,262162,How many chairs are in the photo?,262162018,number,"[{'answer': '1', 'answer_confidence': 'yes', '...",1,how many
39,262162,How many seats are there?,262162021,number,"[{'answer': '1', 'answer_confidence': 'yes', '...",1,how many
40,262162,How many frames are on the wall?,262162022,number,"[{'answer': '7', 'answer_confidence': 'yes', '...",7,how many
47,262162,How many pictures on the wall?,262162029,number,"[{'answer': '7', 'answer_confidence': 'yes', '...",7,how many
67,131115,How many people are on the field?,131115002,number,"[{'answer': '3', 'answer_confidence': 'yes', '...",3,how many people are


In [ ]:
d[['question','answers','multiple_choice_answer']].head()

In [ ]:
d['question'].str.lower().str.split()

In [ ]:
results = set()
d['question'].apply(process_punctuation).str.lower().str.split().apply(results.update)

In [40]:
results = set()
d['answers'].apply(lambda x: [an for y in x for an in process_punctuation(y['answer'].lower().strip()).split()]).apply(results.update)

0         None
1         None
2         None
3         None
4         None
5         None
6         None
7         None
8         None
9         None
10        None
11        None
12        None
13        None
14        None
15        None
16        None
17        None
18        None
19        None
20        None
21        None
22        None
23        None
24        None
25        None
26        None
27        None
28        None
29        None
          ... 
214324    None
214325    None
214326    None
214327    None
214328    None
214329    None
214330    None
214331    None
214332    None
214333    None
214334    None
214335    None
214336    None
214337    None
214338    None
214339    None
214340    None
214341    None
214342    None
214343    None
214344    None
214345    None
214346    None
214347    None
214348    None
214349    None
214350    None
214351    None
214352    None
214353    None
Name: answers, Length: 214354, dtype: object

In [38]:
nested = [[1,2],[3,4],[5,6]]
[x for sublist in nested for x in sublist]
np.array(nested).ravel()

array([1, 2, 3, 4, 5, 6])

In [15]:
for ans in d['answers']:
    for a in ans:
        if not a['answer']:
            print(ans)

In [41]:
results

{'7:08',
 'sees',
 'noisy',
 'survival',
 'scrapers',
 'aerobie',
 'connections',
 'jumpin',
 'footlockers',
 'adams',
 'lakeside',
 'brushing',
 'toad',
 'movements',
 'lifetime',
 'acacia',
 'blues',
 'slop',
 '22000',
 'tyrannosaurus',
 '8w',
 'punched',
 'cawood',
 '500w',
 'aimm',
 'hexagons',
 'fl',
 'missiles',
 'masher',
 '2.45',
 'lieh',
 'watchers',
 '1232',
 'obstruct',
 'pounds',
 '7147',
 '006384',
 '228',
 'panama',
 'saplings',
 'animation',
 'santas',
 'computers',
 'motorcyclists',
 'fans',
 'aged',
 'parts',
 'drape',
 'chunked',
 'campaign',
 'preparation',
 'climbed',
 'powered',
 'gazelles',
 'c92',
 'stilts',
 'railways',
 'underbelly',
 'm&',
 '34th',
 'bruising',
 '100',
 '901',
 'columbus',
 'sides',
 'cupboards',
 'path',
 'footprint',
 'l.92',
 'corde',
 'statesman',
 'reunion',
 'menswear',
 'malaysia',
 'catholics',
 'splintered',
 'dumbbell',
 'placeholders',
 'plume',
 'park',
 'teapot',
 'sbarre',
 'meat',
 'keli',
 'flosser',
 'happens',
 'beach',
 'jew

In [53]:
dict(enumerate(results,start=3))

{3: '7:08',
 4: 'sees',
 5: 'noisy',
 6: 'survival',
 7: 'scrapers',
 8: 'aerobie',
 9: 'connections',
 10: 'jumpin',
 11: 'footlockers',
 12: 'adams',
 13: 'lakeside',
 14: 'brushing',
 15: 'toad',
 16: 'movements',
 17: 'lifetime',
 18: 'acacia',
 19: 'blues',
 20: 'slop',
 21: '22000',
 22: 'tyrannosaurus',
 23: '8w',
 24: 'punched',
 25: 'cawood',
 26: '500w',
 27: 'aimm',
 28: 'hexagons',
 29: 'fl',
 30: 'missiles',
 31: 'masher',
 32: '2.45',
 33: 'lieh',
 34: 'watchers',
 35: '1232',
 36: 'obstruct',
 37: 'pounds',
 38: '7147',
 39: '006384',
 40: '228',
 41: 'panama',
 42: 'saplings',
 43: 'animation',
 44: 'santas',
 45: 'computers',
 46: 'motorcyclists',
 47: 'fans',
 48: 'aged',
 49: 'parts',
 50: 'drape',
 51: 'chunked',
 52: 'campaign',
 53: 'preparation',
 54: 'climbed',
 55: 'powered',
 56: 'gazelles',
 57: 'c92',
 58: 'stilts',
 59: 'railways',
 60: 'underbelly',
 61: 'm&',
 62: '34th',
 63: 'bruising',
 64: '100',
 65: '901',
 66: 'columbus',
 67: 'sides',
 68: 'cupboa

In [ ]:
from collections import Counter
Counter(d['question'])

In [ ]:
class MyDataset(Dataset):
    def __init__(self):
        question = pd.DataFrame(ques['questions'])
        answer = pd.DataFrame(ans['annotations'])
        self.input = question.merge(answer,how="left",on=['question_id','image_id'])
    def __len__(self):
        return len(self.input)
    def __getitem__(self,idx):
#         ques_typ = self.input['question_type'][idx]
#         question = self.input['question'][idx]
#         answer = self.input['answers'][idx]
        print(idx,str(self.input['image_id'][idx]))
        image = Image.open('../data/raw/val2014/COCO_val2014_000000%06d.jpg'%self.input['image_id'][idx])
        return get_transform(448)(image)

In [ ]:
dataset = MyDataset()

In [ ]:
dataset[0]

In [ ]:
list(torch.utils.data.DataLoader(dataset))[0]

In [ ]:
"{:06d}".format(42)

In [ ]:
""%420

In [79]:
#MAx_LEN
d[['question','multiple_choice_answer']].apply(lambda x:len(x['question']) if len(x['question'])>len(x['multiple_choice_answer']) else len(x['multiple_choice_answer']) ,axis=1).max()

100

In [109]:
for z in d:
    print(z)
    break

image_id


In [112]:
d['multiple_choice_answer'].apply(lambda x:len(x.split())).max()

12

In [115]:
d[['question','multiple_choice_answer']].apply(lambda x:max(map(len,map(str.split,x))),axis=1).max()

23

In [97]:

max(map(len,map(str.split,('hi gh','kl'))))

2

In [91]:
max(len,[[1,2],[2,3,5]])

TypeError: '>' not supported between instances of 'list' and 'builtin_function_or_method'

In [ ]:
max(*map(len,(['hi','gh'],['kl'])))